In [ ]:
#Fine Tuning
#https://platform.openai.com/docs/guides/fine-tuning

In [3]:
import os
from openai import OpenAI
import pandas as pd
import json

In [4]:
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY", "<your key>"))

In [16]:
training_data = pd.read_csv("training_data.csv")
training_data.head(3)

,Prompt,Completion
0,"The sun is shining brightly, filling the day w...",positive
1,I am surrounded by loving and supportive frien...,positive
2,Each day brings new opportunities for growth a...,positive


In [8]:
prompts = training_data["Prompt"]

In [9]:
completions = training_data['Completion']

In [12]:
#transporm to json format
#Example:
#[{'prompt': 'The sun is shining brightly, filling the day with warmth and joy. ->',  'completion': 'positive'},...]
json_format = [{"prompt":prompt,"completion":completion} for prompt,completion in zip(prompts,completions)]

In [13]:
json_format[1]

{'prompt': 'I am surrounded by loving and supportive friends who lift me up. ->',
 'completion': 'positive'}

In [14]:
with open("training_data.json", "w") as f:
    for item in json_format:
        f.write(json.dumps(item))
        f.write('\n')

In [ ]:
#data prep
#move to cmd
> pip install --upgrade openai

> set OPEN_API_KEY=<your key>

# here yoy will be asked several questions, for the purpose of this lesson we need just file in JSONL format:
# Based on the analysis we will perform the following actions:
# - [Necessary] Your format `JSON` will be converted to `JSONL`
# - [Recommended] Add a whitespace character to the beginning of the completion [Y/n]: n
# - [Recommended] Would you like to split into training and validation set? [Y/n]: n
# Your data will be written to a new JSONL file. Proceed [Y/n]: y
> openai tools fine_tunes.prepare_data -f "training_data.json"

In [18]:
#Upload a training file - https://platform.openai.com/docs/guides/fine-tuning/upload-a-training-file
#You an find file in Storage section and also manualy upload it there - https://platform.openai.com/storage
client.files.create(
  file=open("training_data_prepared.jsonl", "rb"),
  purpose="fine-tune"
)

FileObject(id='file-3eONCRf0WmuyrzhRrhX2sXgA', bytes=4122, created_at=1714220047, filename='training_data_prepared.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

In [23]:
#Create a fine-tuned model - https://platform.openai.com/docs/guides/fine-tuning/create-a-fine-tuned-model
#You can visualise job in Fine-tuning ection and also manualy create this job - https://platform.openai.com/finetune
client.fine_tuning.jobs.create(
  training_file="file-WPXe3jZperw2jy9JSYSDseBD", #File ID
  model="babbage-002"
)

FineTuningJob(id='ftjob-0VAVEP2HH7q3YKWE1Rg4ot4x', created_at=1714220693, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='babbage-002', object='fine_tuning.job', organization_id='org-xF8zUKwOmUKJwbqibY7GugZu', result_files=[], seed=611352198, status='validating_files', trained_tokens=None, training_file='file-WPXe3jZperw2jy9JSYSDseBD', validation_file=None, integrations=[], user_provided_suffix=None, estimated_finish=None)

In [24]:
#The name can be fund in Fine-tunning section
fine_tuned_model_name = "ft:babbage-002:personal::9IbQzK54"

In [26]:
response = client.completions.create(
model=fine_tuned_model_name,
prompt="Not too thrilled with the blog ->",
    max_tokens=1,
    temperature=0
)

print(response.choices[0].text.strip())

negative
